In [ ]:
pip install mahotas

     |████████████████████████████████| 5.7MB 12.1MB/s 


In [ ]:

#-----------------------------------
# GLOBAL FEATURE EXTRACTION
#-----------------------------------
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import mahotas 
import cv2
import os
import h5py

import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -uq "/content/drive/My Drive/apple.zip" -d "/content/drive/My Drive/strawberries"

In [ ]:
# get the training labels
train_path = "/content/drive/My Drive/apples"
train_labels = os.listdir(train_path)
images_per_class       = [625,615,270,1640]#apple total 630,621,275,1645
image_per_folder       = {
    "Apple___Black_rot"         : 615,#total 621
    "Apple___Cedar_apple_rust"  : 270,#total 275
    "Apple___healthy"           : 1640,#total 1645
    "Apple___Apple_scab"        : 625,#total 630
}
fixed_size             = tuple((256, 256))
bins                   = 8

In [ ]:
train_labels

['Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Apple___Apple_scab']

In [ ]:
# train_labels.remove('Leaf_blight')
# train_labels.remove('Black_rot')
# train_labels.remove('Apple___Cedar_apple_rust')

In [ ]:
train_labels

['Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Apple___Apple_scab']

In [ ]:
# Converting each image to RGB from BGR format

def rgb_bgr(image):
    rgb_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return rgb_img

In [ ]:
# Conversion to HSV image format from RGB

def bgr_hsv(rgb_img):
    hsv_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2HSV)
    return hsv_img

In [ ]:
# image segmentation

# for extraction of green and brown color


def img_segmentation(rgb_img,hsv_img):
    lower_green = np.array([25,0,20])
    upper_green = np.array([100,255,255])
    healthy_mask = cv2.inRange(hsv_img, lower_green, upper_green)
    result = cv2.bitwise_and(rgb_img,rgb_img, mask=healthy_mask)
    lower_brown = np.array([10,0,10])
    upper_brown = np.array([30,255,255])
    disease_mask = cv2.inRange(hsv_img, lower_brown, upper_brown)
    disease_result = cv2.bitwise_and(rgb_img, rgb_img, mask=disease_mask)
    final_mask = healthy_mask + disease_mask
    final_result = cv2.bitwise_and(rgb_img, rgb_img, mask=final_mask)
    return final_result

In [ ]:
# feature-descriptor-1: Hu Moments
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

In [ ]:
# feature-descriptor-2: Haralick Texture
def fd_haralick(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    return haralick

In [ ]:
# feature-descriptor-3: Color Histogram
def fd_histogram(image, mask=None):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

In [ ]:
#Finding the correct leaf contour from the list of contours
def find_contour(cnts,n,m):
    print(cnts)
    contains = []
    y_ri,x_ri = n,m
    for cc in cnts:
        yn = cv2.pointPolygonTest(cc,(x_ri//2,y_ri//2),False)
        contains.append(yn)
        print(contains[0])
    val = [contains.index(temp) for temp in contains if temp>0]
    print(val)
    print(val[0])
    contains.clear()
    return val[0]


In [ ]:
def create_dataset():
  names = ['isHealthy','mean_r','mean_g','mean_b','stddev_r','stddev_g','stddev_b', \
              'contrast','correlation','inverse_difference_moments','entropy']
  df = pd.DataFrame([], columns=names)
  for training_name in train_labels:
    # join the training data path and each species training folder
    dir = os.path.join(train_path, training_name)

    # get the current training label
    current_label = training_name
  
    # loop over the images in each sub-folder
    for x in range(1,image_per_folder[current_label]):
     
        # get the image file name
        file = dir + "/image (" + str(x) + ")"+ ".JPG"
        print(file)
        # file = dir + "/image (" + "84" + ")"+ ".JPG"
       
        # read the image and resize it to a fixed-size
        img = cv2.imread(file)
        img = cv2.resize(img, fixed_size)

        # # #remove backgroud
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
        # gs = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        # blur = cv2.GaussianBlur(gs, (25,25),0)
        # ret_otsu,im_bw_otsu = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        # kernel = np.ones((20,20),np.uint8)
        # closing = cv2.morphologyEx(im_bw_otsu, cv2.MORPH_CLOSE, kernel)

        # contours, hierarchy = cv2.findContours(closing,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        # print(len(contours))
        # plottedContour = cv2.drawContours(gs,contours,-1,(0,255,0),2)

        # n = img.shape[0]
        # m = img.shape[1]

        # black_img = np.empty([n,m,3],dtype=np.uint8)
        # black_img.fill(0)
        
        # # index = find_contour(contours,n,m)
        # # cnt = contours[index]
        # # contours.clear()
        # # mask = cv2.drawContours(black_img, [cnt] , 0, (255,255,255), -1)  
 
        # mask = cv2.drawContours(black_img, [contours[0]] , 0, (255,255,255), -1)  
      
    
        # maskedImg = cv2.bitwise_and(img, mask)

        # white_pix = [255,255,255]
        # black_pix = [0,0,0]
        # final_img = maskedImg
        # h,w,channels = final_img.shape
        # for x in range(0,w):
        #     for y in range(0,h):
        #         channels_xy = final_img[y,x]
        #         if all(channels_xy == black_pix):    
        #             final_img[y,x] = white_pix
        # plt.imshow(final_img)
    
        #Preprocessing
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        gs = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        blur = cv2.GaussianBlur(gs, (25,25),0)
        ret_otsu,im_bw_otsu = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        kernel = np.ones((50,50),np.uint8)
        closing = cv2.morphologyEx(im_bw_otsu, cv2.MORPH_CLOSE, kernel)

        # plt.imshow(closing)

        # final_img = img

        #get features
        #Color features
        red_channel = img[:,:,0]
        green_channel = img[:,:,1]
        blue_channel = img[:,:,2]
        blue_channel[blue_channel == 255] = 0
        green_channel[green_channel == 255] = 0
        red_channel[red_channel == 255] = 0
        
        red_mean = np.mean(red_channel)
        green_mean = np.mean(green_channel)
        blue_mean = np.mean(blue_channel)
        
        red_std = np.std(red_channel)
        green_std = np.std(green_channel)
        blue_std = np.std(blue_channel)
        
        #Texture features
        textures = mahotas.features.haralick(gs)
        ht_mean = textures.mean(axis=0)
        contrast = ht_mean[1]
        correlation = ht_mean[2]
        inverse_diff_moments = ht_mean[4]
        entropy = ht_mean[8]
        #write in csv

        vector = [training_name ,red_mean,green_mean,blue_mean,red_std,green_std,blue_std,\
                  contrast,correlation,inverse_diff_moments,entropy ]
        df_temp = pd.DataFrame([vector],columns=names)
        df = df.append(df_temp)
    print("[STATUS] processed folder: {}".format(current_label))

  print("[STATUS] completed Global Feature Extraction...")

  return df

In [ ]:
images_per_class[2]


270

In [ ]:

dataset = create_dataset()
dataset.shape
dataset.to_csv("apple_features.csv")
print("[STATUS] completed Global Feature Extraction...")

/content/drive/My Drive/apples/Apple___Black_rot/image (1).JPG
/content/drive/My Drive/apples/Apple___Black_rot/image (2).JPG
/content/drive/My Drive/apples/Apple___Black_rot/image (3).JPG
/content/drive/My Drive/apples/Apple___Black_rot/image (4).JPG
/content/drive/My Drive/apples/Apple___Black_rot/image (5).JPG
/content/drive/My Drive/apples/Apple___Black_rot/image (6).JPG
/content/drive/My Drive/apples/Apple___Black_rot/image (7).JPG
/content/drive/My Drive/apples/Apple___Black_rot/image (8).JPG
/content/drive/My Drive/apples/Apple___Black_rot/image (9).JPG
/content/drive/My Drive/apples/Apple___Black_rot/image (10).JPG
/content/drive/My Drive/apples/Apple___Black_rot/image (11).JPG
/content/drive/My Drive/apples/Apple___Black_rot/image (12).JPG
/content/drive/My Drive/apples/Apple___Black_rot/image (13).JPG
/content/drive/My Drive/apples/Apple___Black_rot/image (14).JPG
/content/drive/My Drive/apples/Apple___Black_rot/image (15).JPG
/content/drive/My Drive/apples/Apple___Black_rot/